In [ ]:
from pyroSAR import identify
name = 'data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip'
scene = identify(name)
print(scene)

In [ ]:
from pyroSAR.auxdata import dem_autoload
from spatialist import Vector

site = 'data/aoi/aoi_tmp/aoi_tmp.shp'
vrt = 'data/mosaic.vrt'

with Vector(site) as vec:
    vrt = dem_autoload(geometries=[vec],
                       demType='SRTM 1Sec HGT',
                       vrt=vrt,
                       buffer=0.1)


In [ ]:
from pyroSAR.snap import geocode
geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
        outdir='data/out', test=True)

In [ ]:
from pyroSAR import Archive
dbfile = 'data/scenes.db'
with Archive(dbfile) as archive:
    archive.insert(scene)


In [ ]:
from spatialist import Vector
archive = Archive('data/scenes.db')
outdir = 'data/out'
maxdate = '20220202'
site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'
with Vector(site_shp) as site:
    selection_proc = archive.select(vectorobject=site,
                                    processdir=outdir,
                                    maxdate=maxdate,
                                    sensor=('S1A', 'S1B'),
                                    product='GRD',
                                    acquisition_mode='IW',
                                    vv=1)
archive.close()

In [ ]:
# test groupsize, see what grousize is needed to get all steps into one group; currently it's 8 steps + read/write == groupsize of 8
# geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
#         outdir='data/out',
#         spacing=10,
#         shapefile=site_shp,
#         polarizations=['VV', 'VH'],
#         scaling='dB',
#         cleanup=True,
#         tmpdir='data/tmp',
#         groupsize=8,
#         test=True)

from pyroSAR.snap.auxil import groupbyWorkers
groupbyWorkers('data/out/raw/S1A__IW___A_20210209T170054_Cal_NR_Orb_TF_Spk_TC_dB_proc.xml', n=8)

In [ ]:
from pyroSAR.snap import geocode
import os


files = [os.path.join('data/download/', each) for each in os.listdir('data/download/') if each.endswith('.zip')]
# satfiles = [os.path.join(root, name)
#             for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021')
#             for name in files
#             if name.endswith("zip")]
# satfiles = satfiles[0:3]
# site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'
site_shp = 'data/aoi/flotow_shape_tmp.geojson'

for single_file in files:
# for single_file in satfiles:
    geocode(infile=single_file,
        outdir='data/out/raw',
        spacing=10,
        shapefile=site_shp,
        polarizations=['VV', 'VH'],
        scaling='dB',
        geocoding_type='Range-Doppler',
        removeS1BorderNoise=False,
        removeS1ThermalNoise=False, #not really necessary for SAR data
        allow_RES_OSV=False,
        demName='SRTM 1Sec HGT',
        externalDEMFile=None,
        terrainFlattening=True,
        speckleFilter='Refined Lee',
        refarea='gamma0', #['sigma0','gamma0']
        test=False,
        cleanup=True,
        tmpdir='data/tmp',
        groupsize=8)



In [ ]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os

tif_files = [file for file in os.listdir('data/out/raw') if file.endswith(".tif")]
# site_shp = 'data/aoi/flotow.shp'
site_shp = 'data/aoi/flotow_shape_tmp.geojson'

# Open the .tif file
for file in tif_files:
    with rasterio.open(os.path.join('data/out/raw', file)) as src:
        # Open the .shp file using geopandas
        shapefile = gpd.read_file(site_shp)

        # Clip the .tif file using the .shp file geometry
        clipped, transform = mask(src, shapefile.geometry, crop=True)
        out_profile = src.profile.copy()
        out_profile.update({'width': clipped.shape[2],'height': clipped.shape[1], 'transform': transform})

        # Save the clipped .tif file
        with rasterio.open(os.path.join('data/out/cut', file), 'w', **out_profile) as dst:
            dst.write(clipped)


In [ ]:
#download sen1 images
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']
site = 'data/aoi/flotow_shape_tmp.geojson'

api = SentinelAPI(username, pw)

In [ ]:
footprint = geojson_to_wkt(read_geojson(site))
products = api.query(
    footprint,
    producttype="GRD",
    date=("20220101", "20220131"),
    platformname="Sentinel-1",
    orbitdirection="Ascending",
    sensoroperationalmode="IW",
)

products_df = api.to_dataframe(products)
products_df = products_df.sort_values('ingestiondate')
# products_df

In [ ]:
api.download_all(products_df.index, directory_path = 'data/download')

In [2]:
from pyroSAR.snap import geocode
import os

# os.listdir('../sen1ost/data/download/SAR/GRD/2021/')
for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021/'):
	for file in files:
		if(file.endswith(".zip")):
			print(os.path.join(root,file))

../sen1ost/data/download/SAR/GRD/2021/08/08/S1A_IW_GRDH_1SDV_20210808T170101_20210808T170126_039141_049E87_377A.zip
../sen1ost/data/download/SAR/GRD/2021/08/26/S1B_IW_GRDH_1SDV_20210826T170018_20210826T170043_028420_036413_0E60.zip
../sen1ost/data/download/SAR/GRD/2021/08/11/S1B_IW_GRDH_1SDV_20210811T052438_20210811T052503_028194_035D14_EEE4.zip
../sen1ost/data/download/SAR/GRD/2021/08/29/S1A_IW_GRDH_1SDV_20210829T052519_20210829T052544_039440_04A8CE_44BF.zip
../sen1ost/data/download/SAR/GRD/2021/08/14/S1B_IW_GRDH_1SDV_20210814T170017_20210814T170042_028245_035EA1_BB6D.zip
../sen1ost/data/download/SAR/GRD/2021/08/02/S1B_IW_GRDH_1SDV_20210802T170017_20210802T170042_028070_035934_0184.zip
../sen1ost/data/download/SAR/GRD/2021/08/05/S1A_IW_GRDH_1SDV_20210805T052518_20210805T052543_039090_049CCC_2E9F.zip
../sen1ost/data/download/SAR/GRD/2021/08/17/S1A_IW_GRDH_1SDV_20210817T052518_20210817T052543_039265_04A2DC_19A8.zip
../sen1ost/data/download/SAR/GRD/2021/08/20/S1A_IW_GRDH_1SDV_20210820T17

In [7]:

[os.path.join(root, file) for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021/') for file in files if file.endswith(".zip")]

['../sen1ost/data/download/SAR/GRD/2021/08/08/S1A_IW_GRDH_1SDV_20210808T170101_20210808T170126_039141_049E87_377A.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/26/S1B_IW_GRDH_1SDV_20210826T170018_20210826T170043_028420_036413_0E60.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/11/S1B_IW_GRDH_1SDV_20210811T052438_20210811T052503_028194_035D14_EEE4.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/29/S1A_IW_GRDH_1SDV_20210829T052519_20210829T052544_039440_04A8CE_44BF.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/14/S1B_IW_GRDH_1SDV_20210814T170017_20210814T170042_028245_035EA1_BB6D.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/02/S1B_IW_GRDH_1SDV_20210802T170017_20210802T170042_028070_035934_0184.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/05/S1A_IW_GRDH_1SDV_20210805T052518_20210805T052543_039090_049CCC_2E9F.zip',
 '../sen1ost/data/download/SAR/GRD/2021/08/17/S1A_IW_GRDH_1SDV_20210817T052518_20210817T052543_039265_04A2DC_19A8.zip',
 '../sen1ost/data/download/SAR/GRD/2021/